In [16]:
import cv2
import mediapipe as mp
import pyautogui
import time

def count_fingers(lst):
    cnt = 0
    thresh = (lst.landmark[0].y * 100 - lst.landmark[9].y * 100) / 2

    if (lst.landmark[5].y * 100 - lst.landmark[8].y * 100) > thresh:
        cnt += 1
    if (lst.landmark[9].y * 100 - lst.landmark[12].y * 100) > thresh:
        cnt += 1
    if (lst.landmark[13].y * 100 - lst.landmark[16].y * 100) > thresh:
        cnt += 1
    if (lst.landmark[17].y * 100 - lst.landmark[20].y * 100) > thresh:
        cnt += 1
    if (lst.landmark[5].x * 100 - lst.landmark[4].x * 100) > 6:
        cnt += 1

    return cnt

def is_wrist_shown(lst):
    return lst.landmark[0].y * 100 < lst.landmark[9].y * 100  # Wrist is significantly higher

def is_five_fingers_down(lst):
    return all(lst.landmark[i].y * 100 > lst.landmark[i - 2].y * 100 for i in [8, 12, 16, 20])

cap = cv2.VideoCapture(0)

drawing = mp.solutions.drawing_utils
hands = mp.solutions.hands
hand_obj = hands.Hands(max_num_hands=1)

start_init = False
prev = -1

while cap.isOpened():
    end_time = time.time()
    ret, frm = cap.read()

    if not ret:
        break

    frm = cv2.flip(frm, 1)
    res = hand_obj.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))

    if res.multi_hand_landmarks:
        hand_keyPoints = res.multi_hand_landmarks[0]
        cnt = count_fingers(hand_keyPoints)

        if prev != cnt:
            if not start_init:
                start_time = time.time()
                start_init = True
            elif (end_time - start_time) > 0.2:
                if cnt == 1:
                    pyautogui.press("right")
                elif cnt == 2:
                    pyautogui.press("left")
                elif cnt == 3:
                    pyautogui.press("up")
                elif cnt == 4:
                    pyautogui.press("down")
                elif cnt == 5:
                    pyautogui.press("space")

                prev = cnt
                start_init = False

        if is_wrist_shown(hand_keyPoints):
            pyautogui.hotkey("ctrl", "+")
            time.sleep(0.2)
        elif is_five_fingers_down(hand_keyPoints):
            pyautogui.hotkey("ctrl", "-")
            time.sleep(0.2)

        drawing.draw_landmarks(frm, hand_keyPoints, hands.HAND_CONNECTIONS)

    cv2.imshow("Hand Gesture Control", frm)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
